In [20]:
import json
import os
import pandas as pd
from pandas.io.json import json_normalize

%reload_ext version_information
%version_information json, pandas

Software versions
Python 3.7.4 64bit [Clang 10.0.1 (clang-1001.0.46.4)]
IPython 7.7.0
OS Darwin 18.7.0 x86_64 i386 64bit
json 2.0.9
pandas 0.25.0
Fri Aug 30 13:30:06 2019 CEST

In [41]:
path= '/Users/alicia/Downloads/Dimentions.txt'
with open(path, 'r') as file:
     data_test = json.loads(file.read())

In [48]:
data_test[0]

{'indicator': 'score1',
 'items': [{'category': 'Food',
   'list': [{'id': 'H.01',
     'text': 'In the past 4 weeks, was there ever no food for you to eat because of a lack of resources to get food?',
     'common_answer': {'overal': 'no', 'male': 'no', 'female': 'no'}},
    {'id': 'H.02',
     'text': 'How often did this happen?',
     'common_answer': {'overal': 'Sometimes',
      'male': 'Sometimes',
      'female': 'Sometimes'}},
    {'id': 'H.03',
     'text': 'In the past 4 weeks, did you go to sleep at night hungry because there was not enough food?',
     'common_answer': {'overal': 'no', 'male': 'no', 'female': 'no'}},
    {'id': 'H.04',
     'text': 'How often did this happen?',
     'common_answer': {'overal': 'sometimes',
      'male': 'rarely',
      'female': 'sometimes'}},
    {'id': 'H.05',
     'text': 'In the past 4 weeks, did you go a whole day and night without eating because there was not enough food?',
     'common_answer': {'overal': 'no', 'male': 'no', 'female'

In [53]:
json_normalize(data_test, 'items', ['category', ['list', 'id', "text"]],record_prefix ='test',errors='ignore')

,testcategory,testlist,testitems,category,list.id.text
0,Food,"[{'id': 'H.01', 'text': 'In the past 4 weeks, ...",NaN,NaN,NaN
1,Water,NaN,"[{'category': 'Water quality', 'list': [{'id':...",NaN,NaN
2,Shelter,NaN,"[{'category': 'Characteristics of dwelling', '...",NaN,NaN
3,Health,NaN,"[{'category': 'Health Status', 'list': [{'id':...",NaN,NaN
4,Education,NaN,[{'category': 'Highest educational Attainment'...,NaN,NaN
5,Energy/Fuel,NaN,"[{'category': 'Cooking fuel', 'list': [{'id': ...",NaN,NaN
6,Sanitation,"[{'id': 'D.07', 'text': 'What toilet facilitie...",NaN,NaN,NaN
7,Relationships,[],NaN,NaN,NaN
8,Clothing,NaN,"[{'category': 'Protection', 'list': [{'id': 'P...",NaN,NaN
9,Violence,NaN,[],NaN,NaN


In [25]:
data = [{'state': 'Florida',
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]
result = json_normalize(data, 'counties', ['state', 'shortname',
                                           ['info', 'governor']])
result

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


In [24]:
result.to_json()

'{"id":{"0":1.0,"1":null,"2":2.0},"name.first":{"0":"Coleen","1":null,"2":null},"name.last":{"0":"Volk","1":null,"2":null},"name.given":{"0":null,"1":"Mose","2":null},"name.family":{"0":null,"1":"Regner","2":null},"name":{"0":null,"1":null,"2":"Faye Raker"}}'

In [36]:
def f(x):
    return (dict({'state':x.state.iloc[0]},**{k:v for k,v in zip(x.name,x.population)}))

json_doc = result.groupby(['name','state']).apply(f).groupby(level=0).apply(lambda x: x.tolist()).to_dict()

print(json.dumps(json_doc, indent=4))

{
    "Broward": [
        {
            "state": "Florida",
            "Broward": 40000
        }
    ],
    "Cuyahoga": [
        {
            "state": "Ohio",
            "Cuyahoga": 1337
        }
    ],
    "Dade": [
        {
            "state": "Florida",
            "Dade": 12345
        }
    ],
    "Palm Beach": [
        {
            "state": "Florida",
            "Palm Beach": 60000
        }
    ],
    "Summit": [
        {
            "state": "Ohio",
            "Summit": 1234
        }
    ]
}


In [27]:
def deep_search(needles, haystack):
    found = {}
    if type(needles) != type([]):
        needles = [needles]

    if type(haystack) == type(dict()):
        for needle in needles:
            if needle in haystack.keys():
                found[needle] = haystack[needle]
            elif len(haystack.keys()) > 0:
                for key in haystack.keys():
                    result = deep_search(needle, haystack[key])
                    if result:
                        for k, v in result.items():
                            found[k] = v
    elif type(haystack) == type([]):
        for node in haystack:
            result = deep_search(needles, node)
            if result:
                for k, v in result.items():
                    found[k] = v
    return found

In [29]:
deep_search(["name", "P3"], data)

{'name': 'Cuyahoga'}